In [ ]:
import gym
from gym import spaces
import numpy as np
import pandas as pd

class BalloonNavigationEnv(gym.Env):
    def __init__(self):
        # Define observation space and action space
        # Balloon position in 3D (long, lat, alt), time, and wind data grid in a 300km radius
        # Wind data grid is represented as a flattened array
        self.g_x = 35
        self.g_y = 35

        self.winds = self.get_init_winds()

        low_obs = np.array([-50, -50, -20, 0] + [-10] * (8000))
        high_obs = np.array([50,50,20,100] + [10] * (8000))
        self.observation_space = spaces.Box(low=low_obs, high=high_obs, dtype=np.float32)
        self.action_space = spaces.Discrete(3) 


    def reset(self):
        # Initialize the environment (e.g., set initial balloon position, time, wind data)
        self.state = [0,0,0,0]
        initial_observation = np.concatenate([
            self.state,
            self.winds
        ])
        return initial_observation


    def get_init_winds(self):
        import pandas as pd
        file_path = 'simple_winds.csv'
        df = pd.read_csv(file_path)
        return df.values
    
    def get_winds(self, x,y,z,t):
        w_x = x//10
        w_y = y //10
        w_z = z // 10
        w_t = t//10       
        return  self.winds[w_x,w_y,w_z,w_t]
    
    def rewards(self,x,y):
        return -(abs(x - self.g_x) + abs(y-self.g_y))

    def step(self, action):
        
        t = self.state[4]+1
        winds = self.get_winds(x,y,z,t)

        z = self.state[2] + action
        x = self.state[0] + winds[0]
        y = self.state[1] + winds[1]
     
        self.state = [x,y,z,t]

        observation = np.concatenate([
            self.state,
            self.get_init_windst(0)
        ])
        info = {}

        return observation, self.reward(x,y), (x==self.g_x and y==self.g_y), info

    def render(self, mode='human'):
        # Visualization code (optional)
        pass

# Register the environment with Gym
gym.register(id='BalloonNavigation-v0', entry_point='path.to.BalloonNavigationEnv')
env = gym.make('BalloonNavigation-v0')
